In [1]:
pip install transformers

In [2]:
HF_TOKEN =

In [3]:
import os

In [4]:
os.environ["HF_TOKEN"] = HF_TOKEN

In [5]:
import torch

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [8]:
model_name = "google/gemma-3-1b-it"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [11]:
input_conversation = [
    {"role": "user", "content": "Which is the best place to learn GenAI?"},
    {"role": "assistant", "content": "The best place to learn AI is"}
]

In [15]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=True
)
input_tokens

[2,
 105,
 2364,
 107,
 24249,
 563,
 506,
 1791,
 1977,
 531,
 3449,
 8471,
 12553,
 236881,
 106,
 107,
 105,
 4368,
 107,
 818,
 1791,
 1977,
 531,
 3449,
 12498,
 563,
 106,
 107]

In [20]:
input_detokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
    continue_final_message=True,
)
input_detokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is'

In [21]:
output_label = "GenAI Cohort by ChaiCode"
full_conversation = input_detokens + output_label + tokenizer.eos_token
full_conversation

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI isGenAI Cohort by ChaiCode<eos>'

In [22]:
input_tokenized = tokenizer(full_conversation, return_tensors="pt", add_special_tokens=False).to(device)["input_ids"]
input_tokenized

tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,  14696,
          12553, 105657,    632,    684, 119806,   4809,      1]],
       device='cuda:0')

In [23]:
input_ids = input_tokenized[:, :-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print(f"input_ids: {input_ids}")
print(f"target_ids: {target_ids}")

input_ids: tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,  14696,
          12553, 105657,    632,    684, 119806,   4809]], device='cuda:0')
target_ids: tensor([[   105,   2364,    107,  24249,    563,    506,   1791,   1977,    531,
           3449,   8471,  12553, 236881,    106,    107,    105,   4368,    107,
            818,   1791,   1977,    531,   3449,  12498,    563,  14696,  12553,
         105657,    632,    684, 119806,   4809,      1]], device='cuda:0')


In [24]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fct = nn.CrossEntropyLoss()
    cross_entropy = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return cross_entropy

In [14]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to(device)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [25]:
from torch.optim import AdamW
model.train()

optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

for _ in range(10):
    out = model(input_ids=input_ids)
    loss = calculate_loss(out.logits, target_ids).mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(loss.item())

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


12.6875
4.75
2.640625
1.5390625
0.9296875
0.671875
0.59765625
0.5546875
0.51953125
0.484375


In [26]:
input_prompt = [
    {"role":"user", "content": "which is the best place to learn GenAI?"}
]
input = tokenizer.apply_chat_template(
    conversation=input_prompt,
    return_tensors="pt",
    tokenize=True
).to(device)
input

tensor([[     2,    105,   2364,    107,   7650,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107]],
       device='cuda:0')

In [27]:
output = model.generate(input, max_new_tokens=25)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['user\nwhich is the best place to learn GenAI?\nmodel\nThe best place to learn AI isGenAI Cohort by ChaiCode']
